## Importing Data

In [0]:
import pandas as pd
import numpy as np

In [4]:
act_labels = np.array(["walk", "stand", "jog", "sit", "bike", "ups", "downs",
                       "type", "write", "coffee", "talk", "smoke", "eat"])

columns_labels = np.array(["timeStamp", "accX", "accY", "accZ", "linX", "linY","linZ",
                           "gyrX", "gyrY", "gyrZ", "magX", "magY", "magZ", "actLabel"])


wrist = pd.DataFrame(columns = columns_labels)
tmp = pd.read_csv("smartphoneatwrist.csv", header=None)
tmp.columns = columns_labels
wrist = wrist.append(tmp)

wrist = wrist.drop(columns=['timeStamp'])

wrist['actLabel'] -= 11111

print(wrist.shape)
print([(i,v) for i, v in enumerate(act_labels)])

(1170000, 13)
[(0, 'walk'), (1, 'stand'), (2, 'jog'), (3, 'sit'), (4, 'bike'), (5, 'ups'), (6, 'downs'), (7, 'type'), (8, 'write'), (9, 'coffee'), (10, 'talk'), (11, 'smoke'), (12, 'eat')]


In [5]:
act_labels = np.array(["walk", "stand", "jog", "sit", "bike", "ups", "downs",
                       "type", "write", "coffee", "talk", "smoke", "eat"])

sensors = ["accX", "accY", "accZ", "gyrX", "gyrY", "gyrZ","magX", "magY", "magZ"]
sensors.append("actLabel")
red_act_labels = np.array(["walk", "stand", "jog", "sit", "bike", "ups", "downs",
                       "type", "write", "smoke", "eat"])
# red_act_labels = act_labels
activites = [i for i, x in enumerate(act_labels) if x in red_act_labels]
activites

[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12]

In [6]:
wrist  = wrist[wrist["actLabel"].isin(activites)][sensors]
wrist["actLabel"] = wrist["actLabel"].astype("category").cat.codes

tmp = wrist["actLabel"].values
tmp[tmp==11] = 9
tmp[tmp==12] = 10
wrist["actLabel"] = tmp

for i in zip(wrist.actLabel.unique(), red_act_labels):
  print(i)
  
wrist.shape

(0, 'walk')
(1, 'stand')
(2, 'jog')
(3, 'sit')
(4, 'bike')
(5, 'ups')
(6, 'downs')
(7, 'type')
(8, 'write')
(9, 'smoke')
(10, 'eat')


(990000, 10)

### Making Training _ Testing dataset

In [7]:
columns_labels = sensors
wrist_train_ts = pd.DataFrame(columns = columns_labels)
wrist_test_ts = pd.DataFrame(columns = columns_labels)


train_test_ratio = .80
segment = 5
start = 0
step = (int)(((len(wrist)//len(act_labels) ) // segment))
end = step
cut = (int)(train_test_ratio*(end-start))
for i,_ in enumerate(act_labels):
    for j in range(segment):        
        wrist_train_ts = wrist_train_ts.append(wrist[start:cut])  
        wrist_test_ts = wrist_test_ts.append(wrist[cut:end]) 

        start = end
        end += step
        cut = start+(int)(train_test_ratio*(end-start))
        
wrist_train_ts.shape, wrist_test_ts.shape

((791960, 10), (197990, 10))

### Running Sliding Window

In [0]:
def ts_to_secs(values, w, s, standardize = False, **options):
    
    data = values[:,:values.shape[1]-1]    
    act_labels = values[:,values.shape[1]-1]
    mean = 0
    std = 1
    
    data = np.array(data, dtype=np.float64)

    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            std = options.get("std")
            print("[INFO] -- Test Data has been standardized")
        else:
            mean = data.mean(axis=0)
            std = np.std(data, axis=0)
            print("[INFO] -- Training Data has been standardized: the mean is = "+str(mean)+" ; and the std is = "+str(std))            

        data -= mean
        data /= std
    else:
        print("[INFO] -- Without Standardization.....")

    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T

    m = data.shape[0]   # Data Dimension 
    ttp = data.shape[1] # Total Time Points
    number_of_secs = int(round(((ttp - w)/s)))

    ##  Create a 3D matrix for Storing Sections  
    secs_data = np.zeros((number_of_secs , m , w ))
    act_secs_labels = np.zeros(number_of_secs)

    k=0
    for i in range(0 , ttp-w, s):
        j = i // s
        if j >= number_of_secs:
            break

        if act_labels[i] != act_labels[i+w-1]: 
            continue
            
        secs_data[k] = data[:, i:i+w]
        act_secs_labels[k] = act_labels[i]
        k = k+1
        
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    return secs_data, act_secs_labels, mean, std
##________________________________________________________________

In [9]:
w = 100
s = 10
wrist_train_data, wrist_act_train, wrist_train_mean, wrist_train_std = ts_to_secs(wrist_train_ts.values.copy(),
                                                                   w,
                                                                   s,
                                                                   standardize = True)

s = 25
wrist_test_data, wrist_act_test, _, _ = ts_to_secs(wrist_test_ts.values.copy(),
                                                              w,
                                                              s,
                                                              standardize = True,
                                                              mean = wrist_train_mean, 
                                                              std = wrist_train_std)

print("Shape of wrist_train_data: "+str(wrist_train_data.shape))
print("Shape of wrist_test_data:  "+str(wrist_test_data.shape))

[INFO] -- Training Data has been standardized: the mean is = [ 4.91083713e+00 -4.31141898e+00 -3.56024941e+00 -1.55568460e-02
  1.92573487e-02  9.43957687e-03 -1.38038766e+01  1.37777186e+01
  1.60786221e+01] ; and the std is = [ 3.85822703  5.91729175  3.81122627  0.78177491  0.82447662  1.15599653
 22.55622994 21.91590316 23.0503767 ]
[INFO] -- Test Data has been standardized
Shape of wrist_train_data: (79087, 9, 100)
Shape of wrist_test_data:  (7876, 9, 100)


## Training

In [10]:
import tensorflow as tf 
import keras 
import keras.backend as K

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from collections import Counter

from keras import regularizers
from keras.layers import Reshape
from keras.models import Sequential, Model, load_model, model_from_json 
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Reshape, Concatenate,  Dropout , LSTM, TimeDistributed, RepeatVector
from keras.layers import LSTM, TimeDistributed, RepeatVector, ConvLSTM2D, Conv3D, MaxPooling3D, Conv3DTranspose
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, UpSampling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.noise import AlphaDropout

Using TensorFlow backend.


In [0]:
class Estimator:
    l2p = 0.0001
    @staticmethod
    def early_layers(inp, fm, hid_act_func="relu"):
        # Start
        x = Conv2D(256, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # 1
        x = Conv2D(128, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)

        return x
    
    @staticmethod
    def late_layers(inp, num_classes, fm, act_func="softmax", hid_act_func="relu", b_name="Identifier"):
        # 2
        x = Conv2D(128, (1,5), padding="same", strides= (1,1), kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # 2
        x = Conv2D(128, (1,5), padding="same", strides= (3,1), kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # End
        x = Flatten()(x)
        x = Dense(256, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(64, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_classes, activation=act_func, name = b_name)(x)

        return x
   
    @staticmethod
    def build(height, width, num_classes, name, fm, act_func="softmax",hid_act_func="relu"):
        inp = Input(shape=(height, width, 1))
        early = Estimator.early_layers(inp, fm, hid_act_func=hid_act_func)
        late  = Estimator.late_layers(early, num_classes, fm, act_func=act_func, hid_act_func=hid_act_func)
        model = Model(inputs=inp, outputs=late ,name=name)
        return model

In [12]:
## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
wrist_train_data = np.expand_dims(wrist_train_data,axis=3)
print("[INFO] -- Shape of wrist_train_data:", wrist_train_data.shape)
wrist_test_data = np.expand_dims(wrist_test_data,axis=3)
print("[INFO] -- Shape of wrist_test_data", wrist_test_data.shape)

wrist_act_train_labels = to_categorical(wrist_act_train)
wrist_act_test_labels = to_categorical(wrist_act_test)
wrist_act_train_labels.shape, wrist_act_test_labels.shape

[INFO] -- Shape of wrist_train_data: (79087, 9, 100, 1)
[INFO] -- Shape of wrist_test_data (7876, 9, 100, 1)


((79087, 11), (7876, 11))

In [0]:
def print_results(M, X, Y, position):
    result1 = M.evaluate(X, Y, verbose = 2)
    act_acc = result1[1].round(4)*100
    print("***[RESULT]*** "+position+" ACT Accuracy: "+str(act_acc))

    preds = M.predict(X)
    preds = np.argmax(preds, axis=1)
    conf_mat = confusion_matrix(np.argmax(Y, axis=1), preds)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print("***[RESULT]*** "+position+" ACT  Confusion Matrix")
    print(" | ".join(red_act_labels))
    print(np.array(conf_mat).round(3)*100)  

    f1act = f1_score(np.argmax(Y, axis=1), preds, average=None).mean()
    print("***[RESULT]*** "+position+" ACT Averaged F-1 Score : "+str(f1act*100))
    

In [0]:

def eval_act(X,Y, Xt, Yt, act_class_numbers, fm, ep=50, position = "wrist"):
    height = X.shape[1]
    width = X.shape[2]
    ## Callbacks
    eval_metric= "val_acc"    
    #eval_metric = "acc"
    early_stop = keras.callbacks.EarlyStopping(monitor = eval_metric, mode = 'max', patience = 20)
    filepath=position+".hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor=eval_metric, verbose=1, save_best_only=True, mode='max')
    callbacks_list = [early_stop,checkpoint]
    
    eval_act = Estimator.build(height, width, act_class_numbers, name ="EVAL_ACT", fm=fm, act_func="softmax",hid_act_func="relu")
    eval_act.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc'])
    X , Y = shuffle(X,Y)
    print(eval_act.summary())

## Uncomment this to train your own Classifier model
#     eval_act.fit(X, Y,
#                 validation_data = (Xt,Yt),
#                 epochs = ep,
#                 batch_size = 128,
#                 verbose = 2,
#                 callbacks = callbacks_list
#                )

    eval_act.load_weights(position+".hdf5")
    eval_act.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc'])
    
    print_results(eval_act, X, Y, position)

In [15]:
print("Training on Raw Data:")
eval_act(wrist_train_data.copy(), wrist_act_train_labels.copy(),
         wrist_test_data.copy(), wrist_act_test_labels.copy(),
         len(red_act_labels), fm = (3,5), position = "Classifier_model")


Training on Raw Data:







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "EVAL_ACT"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9, 100, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 100, 256)       4096      
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 100, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 50, 256)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 50, 256)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 50, 12

In [16]:
print("Inference on Raw Data:")
from keras.models import load_model
eval_act = load_model("Classifier_model.hdf5")

X = wrist_test_data
Y = wrist_act_test_labels

print_results(eval_act, X, Y, "Classifier_model")

Inference on Raw Data:
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
***[RESULT]*** Classifier_model ACT Accuracy: 99.19
***[RESULT]*** Classifier_model ACT  Confusion Matrix
walk | stand | jog | sit | bike | ups | downs | type | write | smoke | eat
[[ 97.5   0.    0.    0.    0.    0.7   1.5   0.    0.    0.3   0. ]
 [  0.   99.4   0.    0.    0.    0.    0.    0.    0.    0.6   0. ]
 [  0.    0.  100.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.   98.6   0.    0.    0.    1.    0.1   0.1   0.1]
 [  0.    0.    0.    0.  100.    0.    0.    0.    0.    0.    0. ]
 [  0.4   0.1   0.4   0.    0.   98.8   0.    0.    0.    0.3   0. ]
 [  0.    0.    0.    0.    0.    0.3  99.7   0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0.    0.    0.  100.    0.    0.    0. ]
 [  0.    0.    0.    0.    0.    0.    0.    0.   99.9   0.    0.1]
 [  0.    0.    0.    0.1   0.    0.    0.    0.    0.   97.5   2.3]
 [  0.    0.

# Replacement

In [0]:
train_data = wrist_train_data
test_data = wrist_test_data
act_train = wrist_act_train
act_test = wrist_act_test

In [0]:
train_data = np.expand_dims(train_data, axis=1)
test_data = np.expand_dims(test_data, axis=1)

In [19]:
for i,v in enumerate(red_act_labels):
  print(i,v)

0 walk
1 stand
2 jog
3 sit
4 bike
5 ups
6 downs
7 type
8 write
9 smoke
10 eat


In [20]:
print("White Listed", end="--> ")
wl = [0,2,4,5,6]
for i in wl:
    print(red_act_labels[i], end="; ")

print("\nGray Listed", end=" --> ")
gl = [1,3]
for i in gl:
    print(red_act_labels[i], end="; ")

bl = [7, 8, 9,10]
print("\nBlack Listed", end="--> ")
for i in bl:
    print(red_act_labels[i], end="; ")

White Listed--> walk; jog; bike; ups; downs; 
Gray Listed --> stand; sit; 
Black Listed--> type; write; smoke; eat; 

In [21]:
w_train_data = train_data[np.isin(act_train,wl)]
g_train_data = train_data[np.isin(act_train,gl)]
b_train_data = train_data[np.isin(act_train,bl)]
print("[INFO] -- Shape of Train Whites :"+str(w_train_data.shape))
print("[INFO] -- Shape of Train Grays :"+str(g_train_data.shape))
print("[INFO] -- Shape of Train Blacks :"+str(b_train_data.shape))

w_test_data = test_data[np.isin(act_test,wl)]
g_test_data = test_data[np.isin(act_test,gl)]
b_test_data = test_data[np.isin(act_test,bl)]
print("[INFO] -- Shape of Train Whites :"+str(w_test_data.shape))
print("[INFO] -- Shape of Train Grays :"+str(g_test_data.shape))
print("[INFO] -- Shape of Train Blacks :"+str(b_test_data.shape))

[INFO] -- Shape of Train Whites :(35980, 1, 9, 100, 1)
[INFO] -- Shape of Train Grays :(14463, 1, 9, 100, 1)
[INFO] -- Shape of Train Blacks :(28644, 1, 9, 100, 1)
[INFO] -- Shape of Train Whites :(3569, 1, 9, 100, 1)
[INFO] -- Shape of Train Grays :(1399, 1, 9, 100, 1)
[INFO] -- Shape of Train Blacks :(2908, 1, 9, 100, 1)


In [0]:
batch_size = 128
num_of_epochs = 50

rnd_idx_train = np.random.choice(g_train_data.shape[0], b_train_data.shape[0], replace=True)
tmp =  g_train_data[rnd_idx_train,:]
b_train_transformed = tmp.copy()

x_train = np.append(w_train_data, g_train_data, axis=0)
x_train = np.append(x_train, b_train_data, axis=0)

x_train_transformed = np.append(w_train_data, g_train_data, axis=0)
x_train_transformed = np.append(x_train_transformed , b_train_transformed, axis=0)

resh = np.prod(w_train_data.shape[1:])

In [23]:
filepath="RAE_model.hdf5"

w = 100; prs = 1; sens = 3; chns = 3 ; num_classes = 11

inp = Input(shape=(prs, sens*chns, w,1))
x = ConvLSTM2D(128,  kernel_size=(3, 5), strides=(1,1), padding='same', 
               return_sequences=True)(inp)
x = BatchNormalization()(x)

x = ConvLSTM2D(64,  kernel_size=(3, 5), strides=(1,1), padding='same', 
               return_sequences=True)(x)
x = BatchNormalization()(x)

x = Conv3DTranspose(64, (3,5,1), padding="same", activation='selu')(x)
x = BatchNormalization()(x)

x = Conv3DTranspose(128, (3,5,1), padding="same", activation='selu')(x)
x = BatchNormalization()(x)

x = Conv3DTranspose(1, (3,5,1), padding="same", activation='linear')(x)

raet = Model(inputs=inp, outputs=x ,name="REPConvLSTM2D")

print(raet.summary())

eval_metric= "loss"    
early_stop = keras.callbacks.EarlyStopping(monitor = eval_metric, mode = 'min', patience = 20)
checkpoint = ModelCheckpoint(filepath, monitor=eval_metric, verbose=1, save_best_only=True, mode='min')
callbacks_list = [early_stop,checkpoint]

raet.compile(optimizer='adam', loss='mse')

# x_train, x_train_transformed = shuffle(x_train, x_train_transformed)

## Uncomment this to train your own RAE model
# raet.fit(x_train , x_train_transformed,
#                 #validation_split = 0.1,
#                 epochs = num_of_epochs,
#                 batch_size = batch_size,
#                 shuffle = True,
#                 verbose = 1,
#                 callbacks = callbacks_list
#                 )

Model: "REPConvLSTM2D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 9, 100, 1)      0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 1, 9, 100, 128)    991232    
_________________________________________________________________
batch_normalization_7 (Batch (None, 1, 9, 100, 128)    512       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 1, 9, 100, 64)     737536    
_________________________________________________________________
batch_normalization_8 (Batch (None, 1, 9, 100, 64)     256       
_________________________________________________________________
conv3d_transpose_1 (Conv3DTr (None, 1, 9, 100, 64)     61504     
_________________________________________________________________
batch_normalization_9 (Batch (None, 1, 9, 100, 64)   

In [24]:
print("Results after Transformation")
act_test_labels = to_categorical(act_test)

rae = load_model("RAE_model.hdf5")

rep_x_test = test_data.copy()
rep_x_test = rae.predict(rep_x_test, verbose=1)

from keras.models import load_model
eval_act = load_model("Classifier_model.hdf5")
X = rep_x_test[:,0,:,:,:]
Y = act_test_labels
print_results(eval_act, X, Y, "Classifier_model")

Results after Transformation
7876/7876 [==============================] - 26s 3ms/step
***[RESULT]*** Classifier_model ACT Accuracy: 62.71
***[RESULT]*** Classifier_model ACT  Confusion Matrix
walk | stand | jog | sit | bike | ups | downs | type | write | smoke | eat
[[ 97.2   0.    0.    0.    0.    0.7   1.9   0.    0.    0.1   0. ]
 [  0.   98.2   0.    0.    0.3   0.    0.    0.    0.    1.5   0. ]
 [  0.    0.  100.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.   96.8   0.3   0.    0.    0.    0.1   2.8   0. ]
 [  0.    0.    0.    0.  100.    0.    0.    0.    0.    0.    0. ]
 [  0.3   0.1   0.4   0.    0.1  98.8   0.    0.    0.    0.3   0. ]
 [  0.    0.    0.    0.    0.    0.3  99.7   0.    0.    0.    0. ]
 [  0.    0.    0.  100.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.   99.3   0.7   0.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.   94.9   0.1   0.    0.    0.    0.    5.    0. ]
 [  0.    0.    0.   99.4   0.4   0.    0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
